In [1]:
%%configure -f
{
    "driverMemory": "2G",
    "executorMemory": "4G",
    "executorCores": 2,
    "numExecutors": 2
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1437,application_1765289937462_1425,pyspark,idle,Link,Link,None,
1469,application_1765289937462_1456,pyspark,idle,Link,Link,None,
1473,application_1765289937462_1459,pyspark,idle,Link,Link,None,
1474,None,pyspark,starting,,,None,


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("query4_1").getOrCreate()
sc = spark.sparkContext

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1475,application_1765289937462_1461,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
crimes_old_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv",\
    header=True,
    inferSchema=True)
crimes_new_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv",\
    header=True,
    inferSchema=True)

crimes_df = crimes_old_df.unionByName(crimes_new_df)

stations_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Police_Stations.csv",\
    header=True,
    inferSchema=True)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
from sedona.spark import *
from pyspark.sql import functions as F
from pyspark.sql.functions import col
import time, gc

# Create/Init Sedona
spark = SedonaContext.create(spark)

# ----------------------------
# Query 4 (DataFrame API)
# ----------------------------

# Rename X,Y -> lon,lat
stations_geo = (
    stations_df
    .withColumnRenamed("X", "lon")
    .withColumnRenamed("Y", "lat")
)

# Filter crimes: remove nulls + "Null Island"
crimes_df_clean = crimes_df.filter(
    F.col("LAT").isNotNull() &
    F.col("LON").isNotNull() &
    ~((F.col("LAT") == 0) & (F.col("LON") == 0))
)

# Keep only needed columns
crimes_geo = crimes_df_clean.select("AREA", "LAT", "LON")

# Create geometries
stations_geo = stations_geo.withColumn("station_geom", ST_Point(col("lon"), col("lat")))
crimes_geo   = crimes_geo.withColumn("crime_geom",   ST_Point(col("LON"), col("LAT")))

# Join on AREA = PREC
joined_df = crimes_geo.alias("c").join(
    stations_geo.alias("s"),
    col("c.AREA") == col("s.PREC"),
    "inner"
)

final_df = joined_df.select(
    col("s.FID").alias("FID"),
    col("s.DIVISION").alias("DIVISION"),
    col("s.LOCATION").alias("LOCATION"),
    col("s.PREC").alias("PREC"),
    col("s.station_geom").alias("station_geom"),
    col("c.crime_geom").alias("crime_geom")
)

with_dist = final_df.withColumn(
    "distance_m",
    ST_DistanceSphere(col("station_geom"), col("crime_geom"))
)

result = (
    with_dist
    .groupBy("DIVISION")
    .agg(
        F.round(F.avg("distance_m"), 3).alias("average_distance"),
        F.count("*").alias("count")
    )
    .orderBy(F.desc("count"))
)

# ----------------------------
# Timing: 3 warm-ups + 5 runs
# ----------------------------
warmups = 3
runs = 5
times = []

for i in range(warmups + runs):
    start = time.time()
    rows = result.collect()   # action
    dt = time.time() - start

    del rows
    gc.collect()

    if i < warmups:
        print(f"Warm-up {i+1}: {dt:.4f} sec")
    else:
        times.append(dt)
        print(f"Run {i-warmups+1}: {dt:.4f} sec")

mean_time = sum(times) / len(times)
print(f"Mean time (excluding warm-ups): {mean_time:.4f} sec")

# ----------------------------
# Explain only at the end
# ----------------------------

joined_df.explain("formatted")

# Optional: show sample output
result.show(5, truncate=False)





FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Warm-up 1: 11.2836 sec
Warm-up 2: 0.0774 sec
Warm-up 3: 0.0768 sec
Run 1: 0.0750 sec
Run 2: 0.0711 sec
Run 3: 0.0648 sec
Run 4: 0.0743 sec
Run 5: 0.0792 sec
Mean time (excluding warm-ups): 0.0729 sec
== Physical Plan ==
AdaptiveSparkPlan (13)
+- BroadcastHashJoin Inner BuildRight (12)
   :- Union (7)
   :  :- Project (3)
   :  :  +- Filter (2)
   :  :     +- Scan csv  (1)
   :  +- Project (6)
   :     +- Filter (5)
   :        +- Scan csv  (4)
   +- BroadcastExchange (11)
      +- Project (10)
         +- Filter (9)
            +- Scan csv  (8)


(1) Scan csv 
Output [3]: [AREA#447, LAT#469, LON#470]
Batched: false
Location: InMemoryFileIndex [s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv]
PushedFilters: [IsNotNull(LAT), IsNotNull(LON), Or(Not(EqualTo(LAT,0.0)),Not(EqualTo(LON,0.0))), IsNotNull(AREA)]
ReadSchema: struct<AREA:int,LAT:double,LON:double>

(2) Filter
Input [3]: [AREA#447, LAT#469, LON#470]
Condition : (((isnotnu